# Viele Dateien

**Inhalt:** Massenverarbeitung von gescrapten Zeitreihen

**Nötige Skills:** Daten explorieren, Time+Date Basics

**Lernziele:**
- Pandas in Kombination mit Scraping
- Öffnen und zusammenfügen von vielen Dateien (Glob)
- Umstrukturierung von Dataframes (Pivot)
- Plotting Level 4 (Small Multiples)

## Das Beispiel

Wir interessieren uns in diesem Notebook für Krypto-Coins.

Die Webseite https://coinmarketcap.com/ führt Marktdaten zu den hundert wichtigsten Coins auf.

Mit einem einfachen Scraper werden wir diese Daten beschaffen.

Der Pfad zum Projektordner heisst `dataprojects/Krypto/`.

Zu jeder Kryptowährung haben wir ein csv-Datenfile im Unterordner `data/`.

## Vorbereitung

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import re

In [ ]:
import glob

In [ ]:
%matplotlib inline

## Daten laden

**In diesem Teil hat es einige neue Funktionen drin, die wir noch nicht kennen. Sie können sich einfach durchklicken.**

### Einlesen 1: ein einzelnes File

Zu jeder Kryptowährung gibt es eine Datei mit den Kursdaten. Wir laden testweise eines davon:

In [ ]:
path = 'dataprojects/Krypto/'

In [ ]:
df = pd.read_csv(path + 'data/Bitcoin.csv')

In [ ]:
df.head(2)

Das DF ist 366 Einträge lang (da zu 365 + 1 Tagen Daten vorliegen)

In [ ]:
len(df)

Theoretisch könnten wir diesen Code jetzt für jede der 100 Währungen manuell wiederholen.

Doch das ist umständlich, wir wollen automatisch alle Währungen laden.

### Einlesen 2: Alle Files auf einmal

**Schritt 1: Liste von Dateinamen erstellen**

Wir starten damit, dass wir das Verzeichnis durchsuchen, in dem alle Kryptowährungs-Daten abgelegt sind.

Dazu benutzen wir `glob`, ein praktisches Tool aus der Standard Library: https://docs.python.org/3/library/glob.html

In [ ]:
filenames = glob.glob(path + 'data/*.csv')

In [ ]:
len(filenames)

Wir haben nun eine Liste von allen Dateinamen. Die ersten zwei Einträge sind:

In [ ]:
filenames[0:2]

**Schritt 2: Liste von Dataframes anhand der Dateinamen**

Nun lesen wir jede einzelne Datei aus der Liste ein - und zwar in eine **Liste von Dataframes**.

Diese Liste nennen wir `dfs[]`. Sie wird 100 Einträge haben, jedes davon ein Dataframe.

In [ ]:
dfs = []

Zum Einlesen benutzen wir list comprehension...

In [ ]:
dfs = [pd.read_csv(filename) for filename in filenames]

In [ ]:
dfs[0].head(2)

**Schritt 3: Zu jedem Dataframe den Währungsnamen hinzufügen**

Die einzelnen Dataframes in der Liste enthalten die Marktdaten. Doch sie enthalten selbst keine Information darüber, zu welcher Kryptowährung die Daten gehören. Wir führen zu dem Zweck in jedes Dataframe noch eine zusätzliche Spalte hinzu mit dem Namen der Währung. 

Praktisch ist zu diesem Zweck die Python-Funktion `zip()`. Sie fügt zwei separate Listen – hier: die Dataframes und die Dateinamen – wie ein Reissverschluss zusammen, so dass wir gleich durch beide aufs Mal loopen können.

In [ ]:
for df, filename in zip(dfs, filenames):
    df['currency'] = filename
    df['currency'] = df['currency'].str.extract('/data/(.+).csv')

In [ ]:
dfs[0].head(2)

**Schritt 4: Die DFs untereinander in ein einziges DF reihen**

Nun fügen wir alle Dataframes zu einem einzigen, sehr langen Dataframe zusammen.

Dazu benutzen wir die Funktion `pd.concat()`: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html

In [ ]:
df_all = pd.concat(dfs, ignore_index=True)

Das resultierende DF ist nun 100x länger als die einzelnen zuvor:

In [ ]:
df_all.shape

In [ ]:
df_all.head(2)

In [ ]:
df_all.tail(2)

In [ ]:
df_all.dtypes

Wir haben nun ein ellenlanges Dataframe. What next?

### Daten arrangieren

Das hängt davon ab, was wir mit den Daten genau tun wollen.

Eine Option wäre: die verschiedenen Währungen miteinander zu vergleichen. Und zwar anhand der Schlusskurse.

Dazu müssen wir das Dataframe leicht umstellen, mit `pivot()`: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [ ]:
df_pivoted = df_all.pivot(index='date', columns='currency', values='close')

Mit `pivot()` haben wir folgendes gemacht:
- Statt im "Long"-Format sind die Daten nun im "Wide"-Format
- Das heisst in diesem Fall: Wir haben es anhand der Währungen rearrangiert (`columns='currency'`)
- Die Währungen stehen jetzt nicht mehr *untereinander*, sondern *nebeneinander*
- Die Info, die wir für jede Währung ausgewählt haben, ist der Schlusskurs `values='Close'`)
- Das DF ist jetzt wieder nur noch 366 Zeilen lang.
- Index jeder Zeile ist ein Datum (`index='date'`)

In [ ]:
df_pivoted.shape

Nun verfügen wir über einen Index, bei dem eine Zeile jeweils einem einzigartigen Zeitpunkt entspricht.

Wir verschönern die Indexspalte noch etwas:

In [ ]:
df_pivoted.rename_axis(None, inplace=True)

In [ ]:
df_pivoted.head(2)

Wir haben nun ein sauber formatiertes Dataframe mit hundert Spalten, die für jede Kryptowährung, sofern sie zum betreffenden Zeitpunkt existierte, einen Handelskurs enthält.

**Grossartig! Wir sind damit bereit für die Datenanalyse. Ab hier sind Sie an der Reihe...**

## Analyse

### Vorbereitung

Remember: Mit welchem Typ von Daten haben wir es hier zu tun...?

... genau: mit einer Zeitreihe.

Um damit zu arbeiten, müssen wir den Text in der Indexspalte in ein echtes Datum umwandeln.

Und wir müssen den Datums-Index aufsteigend sortieren.

Die nächste Frage ist: Wie vergleichen wir diese Kurse? Was sagt es aus, wenn eine Währung an einem bestimmten Tag zu 0,1976 USD gehandelt wurde und eine andere zu 18,66 USD?

### Vergleichbarkeit herstellen

Diverse Dinge würden sich hier anbieten:
- zB `pct_change()` um die Veränderungen in den Kursen zu analysieren
- oder eine indexierte Zeitreihe, die an einem bestimmten Tag bei 100 beginnt

Wir wählen die zweite Variante. Und speichern dazu die erste Zeile separat ab.

Dann teilen wir jede einzelne Zeile im Dataframe durch die erste Zeile. Und speichern als neues DF ab.

Das neue Dataframe müsste nun indexiert auf 100 sein: Alle Währungen starten am gleichen Punkt.

Stimmt das wirklich? Schauen Sie sich die ersten fünf Zeilen an, um sicher zu sein...

### Auswertung

Anhand der letzten Reihe können wir ablesen, wie viel eine Währung in den 366 Tagen an Wert gewonnen oder verloren hat.

Zeigen Sie die letzte Reihe an und speichern sie in einer neuen Series.

Welche zehn Kryptowährungen haben am meisten Wert gewonnen?

Welche haben am meisten Wert verloren?

### Plots

Erstellen Sie einen einfachen Linienplot mit den relativen Kursen aller Währungen (Start bei 100).

Wow, das sind ziemlich viele Linien! Beschränken wir uns mal auf zehn Kryptowährungen statt hundert.

Wir finden dazu eine Liste, die wir praktischerweise von unserem Scraper noch auf die Seite gelegt haben...

... sie ist gespeichert unter: `dataprojects/Krypto/currencies.csv`.

Laden Sie die Datei und zeigen Sie die ersten zehn Zeilen an.

Speichern Sie die ersten zehn Einträge der Spalte "name" separat ab.

Sie können die Spalten im Dataframe mit den relativen Kursen nun anhand dieser Liste filtern.

Plotten Sie nun ausschliesslich die Entwicklung der zehn ausgewählten Währungen.

Was fällt Ihnen auf?

In [ ]:
# 

# Plotting Level 4

**Hier geht es wieder gemütlicher zu und her - Sie können sich durchklicken und müssen nichts programmieren.**

Inhalt dieser Plotting-Lektion:

- Wir lernen, wie "Small Multiples" geht
- Wir sehen, wie man die matplotlib-Funktionen direkt benutzen kann.

Dazu importieren wir erstmal ein paar Libraries...

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

... und laden eine Datei, welche das Ergebnis der bisherigen Übung enthält (die relativen Kurse aller Währungen).

In [ ]:
# df_pivoted_100.to_csv(path + 'Close_indexed_to_100.csv')

In [ ]:
df_relative = pd.read_csv(path + 'Close_indexed_to_100.csv', index_col=0)
df_relative.index = pd.to_datetime(df_relative.index)

### Ein Plot

Starten wir zuerst mal mit einem Plots: Bitcoin.

Technisch gesehen besteht jede Matplotlib-/Pandas-Grafik aus zwei Dingen:
1. Eine "figure", also eine Abbildung, gewissermassen das Blatt Papier
1. Ein "subplot", also der einzelne Plot selbst, pro "Papier" können es auch mehrere sein

In [ ]:
# Wir erstellen beide Dinge in einem Atemzug: figure und subplot
fig, ax = plt.subplots(figsize=(10,6))

# Und füllen den Plot jetzt mit Inhalt:
df_relative['Bitcoin'].plot(title="Bitcoin", ax=ax)

### Zwei Plots

Als nächstes Plotten wir zwei Währungen auf derselben Figure: Bitcoin und Ethereum.

Wir müssen uns dazu erneut zwei Dinge basteln:
1. Eine "figure", also eine Abbildung
1. Diverse "subplots" für die jeweiligen Währungen

Dazu formatieren wir jetzt die x-Achse etwas speziell.

In [ ]:
# Zuerst kreieren wir nur die Figure
fig = plt.figure(figsize=(12,3))

# Danach die einzelnen Subplots
ax1 = fig.add_subplot(1, 2, 1) # total 1 Zeile, total 2 Spalten, Subplot Nr. 1
ax2 = fig.add_subplot(1, 2, 2) # total 1 Zeile, total 2 Spalten, Subplot Nr. 2

# Und schliesslich füllen wir die Subplots mit Inhalt
df_relative['Bitcoin'].plot(title="Bitcoin", ax=ax1)
df_relative['Ethereum'].plot(title="Ethereum", ax=ax2)

# Hier formatieren wir die x-Achse für Plot 1
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m'))
ax1.xaxis.set_minor_locator(ticker.NullLocator())

# Hier formatieren wir die x-Achse für Plot 2
ax2.xaxis.set_major_locator(mdates.MonthLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m'))
ax2.xaxis.set_minor_locator(ticker.NullLocator())

Einige Angaben dazu, wie man Zeitachsen formatieren kann, gibt es hier:
- TickLocators: https://matplotlib.org/examples/ticks_and_spines/tick-locators.html
- TickFormatters: https://matplotlib.org/gallery/ticks_and_spines/tick-formatters.html

### Sehr viele Plots

Nun plotten wir sämtliche Währungen, die keine "NaN"-Werte enthalten, auf einmal.

Dazu isolieren wir erst kurz die letzte Zeile.

In [ ]:
s_last = df_relative.iloc[-1]

Wie viele Währungen sind nicht "NaN"?

In [ ]:
anzahl_charts = s_last.notnull().sum()
anzahl_charts

Wir sortieren unsere Liste der Währungen anhand der letzten Zeile etwas:

In [ ]:
sortierte_waehrungen = s_last[s_last.notnull()].sort_values(ascending=False)
sortierte_waehrungen.head(2)

Und wiederholen dann wiederum dasselbe Vorgehen wie vorher.

In [ ]:
# Eine Abbildung, die gross genug ist
fig = plt.figure(figsize=(15,18))

# Und nun, für jede einzelne Währung einen Subplot:
for i, waehrung in enumerate(sortierte_waehrungen.index):
    
    # einen Subplot kreieren ... ()
    ax = fig.add_subplot(11, 8, i + 1)

    # ... und mit Inhalt füllen
    df_pivoted_100[waehrung].plot(title=waehrung, ax=ax)
    
    # Auf Ticks verzichten wir hier ganz
    ax.xaxis.set_major_locator(ticker.NullLocator())
    ax.xaxis.set_minor_locator(ticker.NullLocator())

Falls wir zusätzlich noch wollen, dass jeder Plot dieselbe y-Achse hat:

In [ ]:
# Eine Abbildung, die gross genug ist
fig = plt.figure(figsize=(15,18))

# Und nun, für jede einzelne Währung:
for i, waehrung in enumerate(sortierte_waehrungen.index):
    
    # einen Subplot kreieren ...
    ax = fig.add_subplot(11, 8, i + 1)

    # ... und mit Inhalt füllen
    df_pivoted_100[waehrung].plot(title=waehrung, ax=ax)
    
    # Auf Ticks verzichten wir hier ganz
    ax.xaxis.set_major_locator(ticker.NullLocator())
    ax.xaxis.set_minor_locator(ticker.NullLocator())
    
    # Hier setzen wir eine einheitliche y-Achse (und schalten sie aus)
    ax.set_ylim([0, 2500])
    ax.yaxis.set_major_locator(ticker.NullLocator())

### Aber es geht auch einfacher...

Ha! Nachdem wir nun alles Manuell zusammengebastelt haben, mit Matplotlib, hier die gute Nachricht:

*Wir können das mit wenigen Codezeilen auch direkt aus der Pandas-Plot()-Funktion haben :-)*

Die entscheidenden Parameter in der `plot()`-Funktion sind hier:
- `subplots=True` (zeichnet small multiples statt alles nur auf einem Chart)
- `sharey=True` (alle Subplots sollen dieselbe y-Achse verwenden)

In [ ]:
axes = df_pivoted_100[sortierte_waehrungen.index].plot(subplots=True,layout=(11, 8), sharey=True, figsize=(15,18))

axes[0,0].xaxis.set_major_locator(ticker.NullLocator())
axes[0,0].xaxis.set_minor_locator(ticker.NullLocator())

# Übung

**In dieser letzten Sektion haben Sie nochmals Gelegenheit, alles zu üben – Daten arrangieren, analysieren, plotten**

Hier schauen wir uns nicht mehr die Handelskurse, sondern die Handelsvolumen an! Also: Wie viel von den einzelnen Kryptowährungen an einem bestimmten Tag gekfauft und verkauft wurde (gemessen in USD).

Schauen Sie sich nochmals das Dataframe `df_all` an, das wir im Verlauf des Notebooks erstellt haben - es enthält alle Informationen, die wir brauchen, ist aber noch relativ unstrukturiert.

In [ ]:
df_all.head(2)

Welche Spalte interessiert uns? Müssen wir noch etwas daran machen?

### Daten arrangieren

Unternehmen Sie die nötigen Schritte, um mit der Spalte arbeiten zu können. Sie sollten am Ende eine Spalte haben, die nicht mehr als Object, sondern als Float formatiert ist.

Tipp: Speichern Sie alle Modifikationen in einer neuen Spalte ab, damit das Original unverändert bleibt.

Nun wollen wir die Daten umgliedern:
- Für jedes Datum wollen wir eine Zeile
- Für jede Kryptowährung eine Spalte
- Wir interessieren uns für die Handelsvolumen

Formatieren Sie die Werte in der Index-Spalte als Datetime-Objekte und sortieren Sie das Dataframe nach Datum.

### Analyse

Wir machen in dieser Sektion einige einfache Auswertungen und repetieren einige Befehle, u.a. aus dem Time Series Sheet.

**Top-10**: Welches waren, im Schnitt, die zehn meistgehandelten Währungen? Liste und Chart.

Welches waren die zehn Währungen, bei denen das Volumen in absoluten Zahlen am meisten geschwankt ist? (Standardabweichung)

Sieht so aus, als wären es dieselben zehn Währungen.

Können wir angeben, welche von ihnen relativ die grössten Schwankungen hatten, also im Vergleich zum Handelsvolumen?

**Bitcoin vs Ethereum**

Erstellen Sie einen Chart mit dem wöchentlichen Umsatztotal von Bitcoin und Ethereum!

Tipp: Resampling benutzen!

In welchem der letzten 12 Monate wurde insgesamt am meisten mit Bitcoin gehandelt? Mit Ethereum?

Wie viel Bitcoin und Ethereum wird im Durchschnitt an den sieben Wochentagen gehandelt? Barchart.

**Small Multiples**: Hier erstellen wir einen Plot, ähnlich wie oben

Kreieren Sie zuerst eine Liste von Währungen:
- Alle Währungen, die am letzten Handelstag einen Eintrag haben
- Sortiert in absteigender Reihenfolge nach dem Handelsvolumen
- Wir wählen nur die zehn grössten aus

Und jetzt: Small Multiples plotten! Überlegen Sie sich:
- Wie viele Subplots braucht es, wie sollen sie angeordnet sein?
- Wie gross muss die Abbildung insgesamt sein?
- Was ist eine sinnvolle Einstellung für die Y-Achse?

Alternative syntax dafür: